# Web Search Agent

In this notebook, we demonstrate how to use `AssistantAgent` and `UserProxyAgent` to perform tasks which require acquiring info from the web:
* discuss an article based on its URL.

AssistantAgent is an LLM-based agent that can write Python code.  As for our example, we’ll use it for scrapping a web site.

Next, UserProxyAgent is an agent that acts as a proxy for a user when executing code written by AssistantAgent. Depending on the setting of human_input_mode, UserProxyAgent can receive feedback from the user for the AssistantAgent. If the human_input_mode is "TERMINATE," then the UserProxyAgent will execute the code and return the execution result of success/failure. If there is user feedback, then UserProxyAgent passes it back to the AssistantAgent.

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install pyautogen and docker:
```bash
pip install pyautogen docker
```

In [3]:
%pip install "pyautogen>=0.2.3" docker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.2/345.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00


## Construct Agents

The assistant agent and the user proxy agent.`human_input_mode` is specified as "TERMINATE" in the user proxy agent, which prompts for human feedback when it receives a "TERMINATE" signal from the assistant agent.

In [11]:
import autogen

config_list = [
    {
        'model': 'gpt-4o',
        'api_key': '<your OpenAI API key here>',
    },
]

llm_config = {
    "timeout": 600,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

In [12]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "web",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
)

## Summarize an article from URL

Invoke the `initiate_chat()` method of the user proxy agent to start the conversation. When the cell below is run, it will prompt to provide feedback after the assistant agent sends a "TERMINATE" signal at the end of the message. On Enter the conversation will terminate.
Before the "TERMINATE" signal, the user proxy agent will try to execute the code suggested by the assistant agent on behalf of the user.

In [13]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""
What this article is about: https://pureai.com/Articles/2024/03/01/autogen.aspx
""",
)

user_proxy (to assistant):


What this article is about: https://pureai.com/Articles/2024/03/01/autogen.aspx


--------------------------------------------------------------------------------
assistant (to user_proxy):

To determine what the article at the provided URL is about, I will write a Python script that fetches the content of the webpage and prints it. This will allow us to analyze the content and understand the topic of the article.

Here is the Python script to fetch and print the content of the webpage:

```python
# filename: fetch_article.py
import requests
from bs4 import BeautifulSoup

def fetch_article(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract the main content of the article
        article_content = soup.find('div', class_='article-content')
        if article_content:
            print(article_con

ChatResult(chat_id=None, chat_history=[{'content': '\nWhat this article is about: https://pureai.com/Articles/2024/03/01/autogen.aspx\n', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'To determine what the article at the provided URL is about, I will write a Python script that fetches the content of the webpage and prints it. This will allow us to analyze the content and understand the topic of the article.\n\nHere is the Python script to fetch and print the content of the webpage:\n\n```python\n# filename: fetch_article.py\nimport requests\nfrom bs4 import BeautifulSoup\n\ndef fetch_article(url):\n    try:\n        response = requests.get(url)\n        response.raise_for_status()  # Check if the request was successful\n        soup = BeautifulSoup(response.content, \'html.parser\')\n        \n        # Extract the main content of the article\n        article_content = soup.find(\'div\', class_=\'article-content\')\n        if article_content:\n            print(article_cont